# HW2: Разреженная аппроксимация Personalized PageRank и InstantEmbedding

В этой тетради мы реализуем  метод InstantEmbedding -- алгоритм, строящий эмбеддинги **отдельных** вершин графа за время, **не зависящее от размера графа**. 

In [399]:
# !pip install numpy
# !pip install scipy
# !pip install scikit-learn
# !pip install torch_geometric

In [400]:
from torch_geometric import datasets

In [401]:
data = datasets.LastFMAsia(root='data/lastfm-asia')[0]
labels = data.y
edges_directed = data.edge_index.T.tolist()


# The graph is undirected, but is stored as a directed one (like all graphs in PyTorch Geometric),
# so each edge appears twice.
print(f'Number of nodes: {len(labels)}')
print(f'Number of edges: {len(edges_directed) // 2}')
print(f'Average node degree: {len(edges_directed) / len(labels):.2f}')
print(f'Number of classes: {len(labels.unique())}')

Number of nodes: 7624
Number of edges: 27806
Average node degree: 7.29
Number of classes: 18


## Часть 1: Разреженная аппроксимация Personalized PageRank

Целевой алгоритм описан в [этой статье](https://mathweb.ucsd.edu/~fan/wp/localpartfull.pdf) -- изучите разделы 2 и 3, а также лемму 5 на странице 14. 

Сначала проделаем некоторую предварительную работу -- преобразуйте представление графа из списка ребер в список смежности:

In [402]:
graph = [[] for _ in range(len(labels))]
for u, v in edges_directed:
    graph[u].append(v)


Воспользуемся стандартным значением гиперпараметра $\alpha=0.15$. Обратите внимание, что в статье используется альтернативное определение Personalized PageRank, которое требует изменения $\alpha$ для получения традиционной версии Personalized PageRank:

In [403]:
alpha_orig = 0.15
alpha = alpha_orig / (2 - alpha_orig)
alpha

0.08108108108108107

Реализуйте ниже разреженную аппроксимацию PPR, используя стандартные библиотеки Python. 

Примечания:
- Время выполнения алгоритма не должно зависеть от размера графа (= не перебирайте все вершины в графе).
- Обратите внимание, что вы можете получить разные результаты в зависимости от порядка обработки вершин.
- Для лучшей производительности мы рекомендуем не добавлять в очередь уже имеющиеся там вершины (не забудьте написать быструю проверку этого условия).

На самом деле алгоритм может быть эффективно реализован и без очереди -- вы можете использовать любой подход.

In [404]:
from collections import deque, defaultdict

In [405]:
def sparse_approx_ppr(graph, s, alpha=alpha, eps=1e-3):
    """
    Compute sparse approximate Personalized PageRank for starting distribution s.
    
    Returns:
        p - a defaultdict (with zero default value) representation of the PPR vector p. The keys are nodes
            and the values are PPR probabilities.
        r - a defaultdict (with zero default value) representation of the r vector at the end of the algorithm.
            The keys are nodes and the values are the probabilities that remain in the r vector for these nodes.
    """

    #Погнали пупупупу
    #Жаль материться нельзя, описания были бы красочнее:3

    p = defaultdict(float)
    r = defaultdict(float)
    
    #Ну тут просто кидаем стартовый вектор s в r
    for v, val in s.items():
        r[v] = val
    
    #Очередь для избранных, йоу
    queue = deque()
    in_queue = set()
    
    #Функция чтобы все избранные попали в очередь, йоу, а я репер.
    def add_to_queue(v):
        if v not in in_queue and r[v] >= eps * len(graph[v]):
            queue.append(v)
            in_queue.add(v)
    
    #Теперь в очереди челики из самого начального вектора
    for v in r:
        add_to_queue(v)
    
    #Пусть будет обозначение u, а то иначе я путаюсь, со своей привычкой называть вершины графа как v и w. Делаем все по методичке.
    #Такс, ну пока в очереди есть избранные челики, то мы шаманим пупупупу:
    while queue:
        u = queue.popleft()
        in_queue.remove(u)
        
        r_u = r[u]
        deg_u = len(graph[u])
        
        if r_u >= eps * deg_u:
            p[u] = p[u] + alpha * r_u
            
            #Тут у нас остаток, половина остатка идет в r_u, а другая половина идет по соседям, йоу, треш:
            leftover = (1.0 - alpha) * r_u
            half_left = leftover / 2.0
            r[u] = half_left
            
            share = 0.0
            if deg_u > 0:
                share = half_left / deg_u
            for v in graph[u]:
                r[v] += share
                if v not in in_queue and r[v] >= eps * len(graph[v]):
                    queue.append(v)
                    in_queue.add(v)
            
            #Проверочка для u
            if r[u] >= eps * deg_u and u not in in_queue:
                queue.append(u)
                in_queue.add(u)
            
    return p, r


In [406]:
import pickle
from tqdm.notebook import tqdm

In [407]:
test_nodes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 100, 200, 300, 329, 400, 500, 518, 524, 600, 700, 800, 900, 1000,
              1351, 1738, 2000, 2376, 2510, 2854, 3000, 3061, 3450, 3530, 3597, 3720, 4000, 4301, 4785, 5000,
              5127,  5392, 6000, 6101, 6407, 7000, 7237]

In [408]:
with open('data/lastfm-asia/sparse_approx_ppr.pickle', 'rb') as file:
    ppr_list_gt = pickle.load(file)

In [409]:
def compute_defaultdict_diff_norm(defaultdict_1, defaultdict_2):
    all_keys = set(list(defaultdict_1.keys()) + list(defaultdict_2.keys()))
    diff_dict = {key: defaultdict_1[key] - defaultdict_2[key] for key in all_keys}
    norm = sum(val ** 2 for val in diff_dict.values())
    
    return norm


In [410]:
for i, v in enumerate(tqdm(test_nodes)):
    s = {v: 1}
    ppr, r = sparse_approx_ppr(graph, s)
    
    assert abs(sum(ppr.values()) + sum(r.values()) - 1) < 1e-10
    
    ppr_gt = ppr_list_gt[i]
    diff_norm = compute_defaultdict_diff_norm(ppr, ppr_gt)
    assert diff_norm < 3e-3


  0%|          | 0/47 [00:00<?, ?it/s]

Теперь у нас есть быстрый способ вычислить разреженную аппроксимацию PPR эмбеддингов для вершин нашего графа. Давайте вычислим все эмбеддинги вершин и используем их в качестве входных данных для логистической регрессии.

Ввиду разреженности эмбеддингов их хранение в матрице будет расточительно: постройте разреженную $n \times n$ матрицу с эмбеддингами вершин, где $i$-я строка является эмбеддингом $i$-ой вершины. Используйте класс [scipy.sparse.csr_array](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_array.html).

In [411]:
import numpy as np
from scipy import sparse

In [412]:
n = len(labels) 
row = []
col = []
data = []

for i in range(n):
    p, r = sparse_approx_ppr(graph, s = {i: 1})
    
    for j, val in p.items():
        row.append(i)
        col.append(j)
        data.append(val)

embeddings = sparse.csr_array((np.array(data), (np.array(row), np.array(col))),shape=(n, n))

In [413]:
nonzero_share =  embeddings.nnz / (embeddings.shape[0] * embeddings.shape[1])
print(f'{nonzero_share:.4f} of values in the embeddings matrix is non-zero.')

0.0046 of values in the embeddings matrix is non-zero.


In [414]:
embeddings_gt = sparse.load_npz('data/lastfm-asia/sparse_embeddings.npz')

In [415]:
assert ((embeddings - embeddings_gt) ** 2).sum() < 100

In [416]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [417]:
labels = labels.numpy()

# Create the same split as in the seminar.
full_idx = np.arange(len(labels))
train_idx, test_idx = train_test_split(full_idx, test_size=0.75, stratify=labels, random_state=0)

In [418]:
embeddings_train = embeddings[train_idx]
labels_train = labels[train_idx]

embeddings_test = embeddings[test_idx]
labels_test = labels[test_idx]

In [419]:
logreg = LogisticRegression(penalty='l2', C=1000, solver='lbfgs', max_iter=1000)
logreg.fit(embeddings_train, labels_train)

LogisticRegression(C=1000, max_iter=1000)

In [420]:
preds = logreg.predict(embeddings_test)

In [421]:
accuracy = (preds == labels_test).mean()
print(f'sparse approx PPR + logistic regression accuracy: {accuracy:.4f}')

sparse approx PPR + logistic regression accuracy: 0.7959


Ожидаемое значение accuracy -- 0.79.


Давайте проверим, возможно ли уменьшить размерность эмбеддингов без потери точности. Воспользуемся подходом из [статьи InstantEmbedding](https://arxiv.org/abs/2010.06992) — в ней описывается простое хеширование для уменьшения размерности эмбеддинга.

Прочитайте статью и реализуйте две функции, которые генерируют хеш-функции из универсальных хеш-семейств. Примечания:
- Вам не обязательно делать ваши универсальные хеш-семейства сложными
- Вы можете предположить, что ваши хэш-функции не будут использоваться с графами, имеющими > 1 миллиарда вершин

In [422]:
import random

In [423]:
def create_hash_function_d(d, seed):
    #Пукает хеш-функцию, которая для целого x возвращает значение в [0, d - 1] - по сути координата
    random.seed(seed)
    prime = 2147483647  #Тип одно из больших простых, которое < 2^31
    a = random.randint(1, prime - 1)
    b = random.randint(0, prime - 1)
    
    def h(x):
        return ((a * x + b) % prime) % d
    pass
    return h


def create_hash_function_sgn(seed):
    #Пукает хеш-функцию, которая возвращает или -1 или +1 - по сути знак
    random.seed(seed)
    prime = 2147483647
    a = random.randint(1, prime - 1)
    b = random.randint(0, prime - 1)
    
    def h_sgn(x):
        val = ((a * x + b) % prime) % 2
        return 1 if val == 0 else -1
    pass
    return h_sgn

In [424]:
hash_function_d = create_hash_function_d(512, seed = 42)
hash_function_sgn = create_hash_function_sgn(seed = 42)

Теперь реализуем функцию, создающую эмбеддинги на основе PPR. Реализуйте «Algoithm 2. InstantEmbedding» из статьи -- еще раз напомним, что время работы алгоритма не должно зависеть от размера графа.

In [425]:
from math import log

In [437]:
def instant_embedding(graph, v, d=512, eps=1e-3, h_d=hash_function_d, h_sgn=hash_function_sgn):
    """
    Create d-dimensional embedding for node v.
    
    Returns:
        w - the embedding - a list with d elements.
    """
    n = len(graph)  
    p, r = sparse_approx_ppr(graph, s = {v: 1.0}, alpha = alpha, eps = eps)
    
    w = [0.0] * d
    for j, val in p.items():
        weight = max(log(val * n), 0.0) 
        coord = h_d(j)
        sign = h_sgn(j)
        w[coord] += sign * weight
        
    return w


In [438]:
embeddings = np.array([instant_embedding(graph, v) for v in tqdm(range(len(graph)))])

100%|██████████| 7624/7624 [00:04<00:00, 1667.34it/s]


Обучим логистическую регрессию на полученных эмбеддингах:

In [441]:
embeddings_train = embeddings[train_idx]
embeddings_test = embeddings[test_idx]

In [442]:
logreg = LogisticRegression(penalty='l2', C=0.01, solver='lbfgs', max_iter=1000)
logreg.fit(embeddings_train, labels_train)
preds = logreg.predict(embeddings_test)
accuracy = (preds == labels_test).mean()
print(f'InstantEmbedding + logistic regression accuracy: {accuracy:.4f}')

InstantEmbedding + logistic regression accuracy: 0.8047


Вы должны получить accuracy выше 0.79. Как можно заметить, нам удалось уменьшить размерность эмбеддинга без существенных потерь точности модели